## Ejercicios de pair programming 11 de enero: ETL 3.1

Es el momento de meter todos nuestros datos en SQL 💪🏽!!! 

En este ejercicio nos crearemos dos tablas en una BBDD creada por nosotras. 

Una de las tablas contendrá la información que obtuvimos de los ejercicios de pair programming de Limpieza, es decir, 

el data set de ataques de tiburones limpito. La segunda tabla tendrá la información obtenida en el ejercicio de pair de ETL 1.


📌 Nota Todo lo tendremos que hacer desde jupyter notebook


Cread la BBDD con el nombre de tiburones.

Cread las tablas de la BBDD:

Tabla ataques

Tabla clima

BONUS Insertar los datos en las tablas.

📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación

In [1]:
#importamos las librerias
import pandas as pd
import mysql.connector

In [8]:
#abrimos el archivo de tiburones limpio
df_tiburon = pd.read_csv('../datos/tiburon8.csv', index_col = 0)
df_tiburon.head(2)

,case_number,year,type,country,activity,injury,mes,deceso,genero,especies,edad
0,2018.06.25,2018,Boating,usa,Paddling,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Swimming,FATAL,Jun,Y,M,Tiger,18.0


In [9]:
#abrimos el archivo del clima
df_clima = pd.read_csv('../datos/meteo_tiburon.csv', index_col= 0)
df_clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,USA


In [4]:
#hacemos una funcion para crear la base de datos
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab"
    )
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)


In [7]:
#llamamos a la funcion y creamos la base de datos 'tiburones'
mybd= crear_bbdd('tiburones')

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones;


In [22]:
#creamos una funcion para crear tabla nueva
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{contraseña}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [46]:
#hacemos la query para la tabla ataques
tabla_ataques = '''
CREATE TABLE IF NOT EXISTS `tiburones`.`ataques` (
  `case_number` VARCHAR(20) NOT NULL ,
  `year`INT NOT NULL,
  `type` VARCHAR(50) NOT NULL,
  `country` VARCHAR(50),
  `activity` VARCHAR (100),
  `injury` TEXT,
  `mes` VARCHAR (10),
  `deceso` VARCHAR(20),
  `genero` VARCHAR (20),
  `especies` VARCHAR (50),
  `edad` FLOAT,
  PRIMARY KEY (`case_number`))
ENGINE = InnoDB;
'''

In [20]:
df_clima.reset_index(inplace= True)   #generamos una columna de index para usarla como primary key en la tabla de clima

In [34]:
#hacemos la query para la tabla clima

tabla_clima = '''
CREATE TABLE IF NOT EXISTS `tiburones`.`clima` (
  `index` INT NOT NULL AUTO_INCREMENT,
  `timepoint` INT,
  `cloudcover` INT, 
  `highcloud` INT, 
  `midcloud` INT, 
  `lowcloud` INT,
  `rh_profile` VARCHAR (100), 
  `wind_profile` VARCHAR(100), 
  `temp2m` INT, 
  `lifted_index` INT, 
  `rh2m` INT,
  `msl_pressure` INT, 
  `prec_type` VARCHAR (10), 
  `prec_amount` INT, 
  `snow_depth` INT,
  `wind10m.direction` INT, 
  `wind10m.speed` INT, 
  `pais` VARCHAR (20),
  PRIMARY KEY (`index`))
ENGINE = InnoDB;
'''

In [47]:
#llamamos a la funcion
crear_insertar_tabla('tiburones', "AlumnaAdalab", tabla_ataques)

In [35]:
#llamamos a la funcion
crear_insertar_tabla('tiburones', "AlumnaAdalab", tabla_clima)

Ya tenemos generada la base de datos con las dos columnas, ataques y clima

Introducimos ahora los datos en las tablas correspondientes

In [48]:
for indice, fila in df_tiburon.iterrows(): # itreamos por el dataframe.
    
    # definimos nuestra query, igual que si lo hicieramos en workbench. ⚠️ Como estamos definiendo nuestra query en varias líneas usamos las triples comillas
    # lo valores que introduciremos serán los del dataframe que estamos iterando, por lo que usaremos los formats de los strings. 
    
    query_tiburon = f"""
            INSERT INTO ataques (case_number, year, type, country, activity, injury, mes, deceso, genero, especies, edad) 
            VALUES ( "{fila['case_number']}", "{fila['year']}", "{fila['type']}", "{fila['country']}", "{fila['activity']}", "{fila['injury']}", "{fila['mes']}", "{fila['deceso']}", "{fila['genero']}", "{fila['especies']}", "{fila['edad']}");
            """
    # una vez definida la query llamamos a la función que nos inserta los datos. 
    crear_insertar_tabla("tiburones", "AlumnaAdalab", query_tiburon)

1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'laceration to left hip", "Oct", "N", "M", "White", "45.0")' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'laceration to left hip", "Oct", "N", "M", "White", "45.0")' at line 2
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'laceration to left forearm", "Sep", "N", "M", "Unspecified", "58.0")' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'laceration to left forearm", "Sep", "N", "M", "Unspecified", "58.0")' at line 2
1064 (42000): You have an error in your SQL syntax; 

KeyboardInterrupt: 